# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

In [1]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs

print("pandas versão", pd.__version__)

pd.options.display.max_rows = 2000

pandas versão 1.0.1


# <font color='black'>trata o dataset total</font>

In [2]:
# caminho dos arquivos...
caminho = "dados_gd/"

In [3]:
# abrindo os arquivos...
arq_form = caminho + "form.csv"
arq_sla = caminho + "sla.csv"
arq_tasks = caminho + "tasks.csv"
arq_rating = caminho + "rating.csv"

df_tasks = pd.read_csv(arq_tasks, sep = ',', dtype = str)
print("linhas = %s , colunas = %s " % df_tasks.shape)

# df_tasks

linhas = 414 , colunas = 23 


In [4]:
# exclui colunas não utilizáveis
lst_del = ['Entidade', 'Usuário', 'Grupo', 
           'Motivo de cancelamento', 'Notificação',
           'Comentário', 'Apoio', 'Data e Hora de atendimento',
           'Status externo', 'Data de agendamento', 'Agendamento',
           'Categoria', 'Grupo responsável', 'Prazo (em segundos)']
df_tasks.drop(lst_del, axis=1, inplace=True)
lst_nomes = ['entidade', 'servico', 'protocolo', 'dt_conclusao', 'dt_criacao',
             'acao', 'encaminhado_para', 'encerrado', 'cancelado']
df_tasks.columns = lst_nomes

In [5]:
# transforma datas do tipo string para tipo data
df_tasks['dt_conclusao']= pd.to_datetime(df_tasks['dt_conclusao'])
df_tasks['dt_criacao']= pd.to_datetime(df_tasks['dt_criacao'])

In [6]:
# verifica a situação de cada protocolo
df = df_tasks[['protocolo', 'encerrado', 'cancelado']].copy()

df.loc[df['encerrado'] == 'false', 'encerrado'] = 0
df.loc[df['encerrado'] == 'true', 'encerrado'] = 1
df.loc[df['cancelado'] == 'false', 'cancelado'] = 0
df.loc[df['cancelado'] == 'true', 'cancelado'] = 1

df['encerrado'] = pd.to_numeric(df['encerrado'])
df['cancelado'] = pd.to_numeric(df['cancelado'])

# df

In [7]:
# agrupa por protocolo, obtem o máximo de encerrado e cancelado
df = df.groupby('protocolo').agg({'encerrado': 'max', 'cancelado': 'max'})

In [8]:
# filtra os protocolos cancelados
df.drop(df[df['cancelado'] == 1].index, inplace=True)

# exclui a coluna de protocolos cancelados
df.drop('cancelado', axis=1, inplace=True)

# cria um dataframe de processos encerrados
df_ence = df[(df['encerrado'] == 1)].copy()

# cria um dataframe de processos em andamento
df_anda = df[(df['encerrado'] == 0)].copy()

# exclui a coluna encerrado dos 2 dataframes
df_ence.drop('encerrado', axis=1, inplace=True)
df_anda.drop('encerrado', axis=1, inplace=True)

In [9]:
print(df_ence.shape)
print(df_anda.shape)

(19, 0)
(104, 0)


In [10]:
# cria o dataframe de encerrados e dataframe de em andamento
lst_ence = df_ence.index.tolist()
lst_anda = df_anda.index.tolist()

# <font color='black'>cria e trata o dataset de protocolos encerrados</font>

In [45]:
# cria dataset de tasks encerradas
df_ence = df_tasks[df_tasks['protocolo'].isin(lst_ence)].copy()

In [46]:
# ordena por entidade, serviço, protocolo, dt_criação
df_ence.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_ence.reset_index(drop = True, inplace = True)

In [47]:
# calcula a quantidade de dias da ação e exclui as datas de criação e de conclusão
df_ence['dias'] = (df_ence['dt_conclusao'] - df_ence['dt_criacao']) / np.timedelta64(1, 'D')
df_ence.drop(['dt_conclusao', 'dt_criacao'], axis=1, inplace=True)

In [48]:
# exclui colunas não utilizáveis
df_ence.drop(['encerrado', 'cancelado'], axis=1, inplace=True)

# df_ence

In [49]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['protocolo', 'entidade', 'servico'])
      .agg(
        lst_acao=('acao', lambda x: tuple(x)),
        lst_encaminhado=('encaminhado_para', lambda x: tuple(x)),
        sum_dias=('dias', sum)).reset_index())

In [51]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['entidade', 'servico', 'lst_acao', 'lst_encaminhado'])
      .agg(
        media_dias=('sum_dias', 'mean')).reset_index())

In [52]:
df_ence.shape

(14, 5)

# <font color='black'>cria o dataset de protocolos em andamento</font>

In [53]:
# cria dataset de tasks em andamento
df_anda = df_tasks[df_tasks['protocolo'].isin(lst_anda)].copy()

In [54]:
# exclui colunas não utilizáveis
df_anda.drop(['encerrado', 'cancelado', 'dt_conclusao'], axis=1, inplace=True)

# exclui as linhas com a coluna ação NULL
df_anda = df_anda[~df_anda['acao'].isnull()].copy()

# df_anda

In [55]:
# ordena por entidade, serviço, protocolo, dt_criação
df_anda.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_anda.reset_index(drop = True, inplace = True)

In [56]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_anda = (df_anda.groupby(['protocolo', 'entidade', 'servico'])
      .agg(lst_acao=('acao', lambda x: tuple(x)))
      .reset_index()).copy()
# df_anda

# <font color='black'>cria um arquivo com recomendações</font>

In [60]:
# analisa cada protocolo em andamento
# procura a sequência do protocolo em andamento para sugerir o restante da sequência

# zera a lista de recomendações
lst_reco = []

# para cada protocolo em andamento procura uma coleção de sequências dos protocolos encerrados
for i in range(len(df_anda)):
    n_acoes = len(df_anda.loc[i, 'lst_acao'])
    df_aux = df_ence[(df_ence['entidade'] == df_anda.loc[i, 'entidade']) & 
                     (df_ence['servico'] == df_anda.loc[i, 'servico']) &
                     (df_ence['lst_acao'].apply(lambda x: x[:n_acoes]) == df_anda.loc[i, 'lst_acao'])]
    df_aux.reset_index(inplace = True)
    
    # zera a variável que vai achar o registro "entidade/serviço" com o menor tempo de conclusão
    # para quando for achado mais de 1 possibilidade
    min_tempo = 9999999999.9
    reco = []
    encaminhado = ''
    
    for k in range(len(df_aux)):
        valor = np.float64(df_aux.loc[k, 'media_dias'])
        if valor < min_tempo:
            min_tempo = valor
            reco = list(df_aux.loc[k, 'lst_acao'])
            encaminhado = df_aux.loc[k, 'lst_encaminhado']
            
    it = {'protocolo': df_anda.loc[i, 'protocolo'],
         'seq_atual': list(df_anda.loc[i, 'lst_acao']),
         'reco_encaminhado_para': encaminhado,
         'reco_seq_recomendada': reco,
         'tempo_medio': min_tempo,
         'reco_acao_idx': len(list(df_anda.loc[i, 'lst_acao']))}
    lst_reco.append(it)  

In [61]:
import json

# grava arquivo de recomendação
y = json.dumps(lst_reco, indent=2, ensure_ascii=False)
print(y)
with open(caminho + 'sugestao.txt', 'w') as outfile:
    json.dump(lst_reco, outfile, indent=2, ensure_ascii=False)

[
  {
    "protocolo": "2020.10-32.612",
    "seq_atual": [
      "Solicitação"
    ],
    "reco_encaminhado_para": [
      "Ouvidoria",
      "Ouvidoria"
    ],
    "reco_seq_recomendada": [
      "Solicitação",
      "Conclusão Atendimento / Arquivamento"
    ],
    "tempo_medio": 0.00407447337962963,
    "reco_acao_idx": 1
  },
  {
    "protocolo": "2020.10-32.613",
    "seq_atual": [
      "Solicitação"
    ],
    "reco_encaminhado_para": [
      "Ouvidoria",
      "Ouvidoria"
    ],
    "reco_seq_recomendada": [
      "Solicitação",
      "Conclusão Atendimento / Arquivamento"
    ],
    "tempo_medio": 0.00407447337962963,
    "reco_acao_idx": 1
  },
  {
    "protocolo": "2020.10-32.614",
    "seq_atual": [
      "Solicitação",
      "Abertura de Processo Administrativo"
    ],
    "reco_encaminhado_para": [
      "Ouvidoria",
      "Ouvidoria",
      "Ouvidoria",
      "Ouvidoria"
    ],
    "reco_seq_recomendada": [
      "Solicitação",
      "Abertura de Processo Administrativo